> This is the notebook that helped me understand why not to use .csv as an extension, since it can't hold information about the types of columns

During reading, pandas defaults to int64, float64 or object, losing any other types, like int8, string and datetime. 

In [1]:
import pandas as pd
import numpy as np
import gc

## Variables

In [2]:
df = pd.read_csv('Types/SIH/RD2012.csv')

# Removing past index
df.drop('Unnamed: 0', axis=1, inplace=True)

df.columns

C:\Users\ricmo\AppData\Local\Temp\ipykernel_24432\3499200186.py:1: DtypeWarning: Columns (62,78,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Types/SIH/RD2012.csv')


Index(['UF_ZI', 'ANO_CMPT', 'MES_CMPT', 'ESPEC', 'CGC_HOSP', 'N_AIH', 'IDENT',
       'CEP', 'MUNIC_RES', 'NASC', 'SEXO', 'UTI_MES_IN', 'UTI_MES_AN',
       'UTI_MES_AL', 'UTI_MES_TO', 'MARCA_UTI', 'UTI_INT_IN', 'UTI_INT_AN',
       'UTI_INT_AL', 'UTI_INT_TO', 'DIAR_ACOM', 'QT_DIARIAS', 'PROC_SOLIC',
       'PROC_REA', 'VAL_SH', 'VAL_SP', 'VAL_SADT', 'VAL_RN', 'VAL_ACOMP',
       'VAL_ORTP', 'VAL_SANGUE', 'VAL_SADTSR', 'VAL_TRANSP', 'VAL_OBSANG',
       'VAL_PED1AC', 'VAL_TOT', 'VAL_UTI', 'US_TOT', 'DT_INTER', 'DT_SAIDA',
       'DIAG_PRINC', 'DIAG_SECUN', 'COBRANCA', 'NATUREZA', 'NAT_JUR', 'GESTAO',
       'RUBRICA', 'IND_VDRL', 'MUNIC_MOV', 'COD_IDADE', 'IDADE', 'DIAS_PERM',
       'MORTE', 'NACIONAL', 'NUM_PROC', 'CAR_INT', 'TOT_PT_SP', 'CPF_AUT',
       'HOMONIMO', 'NUM_FILHOS', 'INSTRU', 'CID_NOTIF', 'CONTRACEP1',
       'CONTRACEP2', 'GESTRISCO', 'INSC_PN', 'SEQ_AIH5', 'CBOR', 'CNAER',
       'VINCPREV', 'GESTOR_COD', 'GESTOR_TP', 'GESTOR_CPF', 'GESTOR_DT',
       'CNES', 'CNPJ_M

In [3]:
dict = {'Coluna':[], 'Espaço_CSV':[]}

memoryUsage = pd.DataFrame(dict)

# Getting the memory usage of the dataframe
for item in df.columns:
    aux = {'Coluna': item, 'Espaço_CSV': [df[item].memory_usage()]}
    df2 = pd.DataFrame(aux)
    memoryUsage = pd.concat([memoryUsage, df2], ignore_index=True)
    
memoryUsage['Espaço_CSV'] = memoryUsage['Espaço_CSV'].astype('Int64')

memoryUsage.loc[:, 'Test_INT'] = pd.NA
memoryUsage.loc[:, 'Test_FLOAT'] = pd.NA
memoryUsage.loc[:, 'Test_CAT'] = pd.NA
memoryUsage.loc[:, 'Test_OBJ'] = pd.NA

memoryUsage.set_index('Coluna', inplace=True)

## Functions

In [6]:
def type_test(df, memoryUsage, col):
    menor_tipo = df[col].dtype
    menor_memo = memoryUsage.loc[col, 'Espaço_CSV']

    # int 
    if df[col].isna().sum() == 0:
        try:
            aux = pd.to_numeric(df[col], downcast='integer')

            a = aux.memory_usage()
            memoryUsage.loc[col, 'Test_INT'] = a

            if menor_memo > a:
                menor_tipo = aux.dtype
                menor_memo = a
        except ValueError:
            print(f'Coluna {col} - int value error')

    # cat
    try: 
        aux = df[col].astype("category")

        a = aux.memory_usage()
        memoryUsage.loc[col, 'Test_CAT'] = a

        if menor_memo > a:
            menor_tipo = aux.dtype
            menor_memo = a
    except ValueError:
        print(f'Coluna {col} - cal value error')

    # float
    try:
        aux = pd.to_numeric(df[col], downcast='float')

        a = aux.memory_usage()
        memoryUsage.loc[col, 'Test_FLOAT'] = a

        if menor_memo > a:
            menor_tipo = aux.dtype
            menor_memo = a
    except ValueError:
        print(f'Coluna {col} - float value error')

    # object
    try: 
        aux = df[col].astype('object')

        a = aux.memory_usage()
        memoryUsage.loc[col, 'Test_OBJ'] = a

        if menor_memo > a:
            menor_tipo = aux.dtype
            menor_memo = a
    except ValueError:
        print(f'Coluna {col} - float value error')

    return menor_tipo
    # df[col] = df[col].astype(menor_tipo)

---

In [10]:
dicionario = {}

for column in df.columns:
    tipo = type_test(df, memoryUsage, column)
    dicionario[column] = tipo

Coluna DIAG_PRINC - int value error
Coluna DIAG_PRINC - float value error
Coluna DIAG_SECUN - float value error
Coluna CID_NOTIF - float value error
Coluna CID_ASSO - float value error
Coluna CID_MORTE - float value error
Coluna REMESSA - int value error
Coluna REMESSA - float value error
Coluna AUD_JUST - float value error
Coluna SIS_JUST - float value error


In [13]:
dicionario

{'UF_ZI': CategoricalDtype(categories=[110000, 110001, 110002, 110003, 110004, 110005, 110006,
                   110008, 110009, 110010,
                   ...
                   522140, 522145, 522150, 522155, 522160, 522170, 522200,
                   522205, 522220, 530000],
 , ordered=False, categories_dtype=int64),
 'ANO_CMPT': CategoricalDtype(categories=[2012], ordered=False, categories_dtype=int64),
 'MES_CMPT': dtype('int8'),
 'ESPEC': dtype('int8'),
 'CGC_HOSP': CategoricalDtype(categories=[   15081000101.0,    29827000128.0,    33940000187.0,
                      38174000658.0,    54015000213.0,    54015000485.0,
                      54015000809.0,    54015001619.0,    54015001708.0,
                      54015001880.0,
                   ...
                   97731889000119.0, 98039795000146.0, 98104193000125.0,
                   98110000000149.0, 98227986000131.0, 98314099000109.0,
                   98339823000140.0, 98416225000128.0, 98591910000190.0,
              

In [8]:
memoryUsage

,Espaço_CSV,Test_INT,Test_FLOAT,Test_CAT,Test_OBJ
Coluna,,,,,
UF_ZI,53386068,26693100,26693100,13425776,53386068
ANO_CMPT,53386068,13346616,26693100,6673490,53386068
MES_CMPT,53386068,6673374,26693100,6673770,53386068
ESPEC,53386068,6673374,26693100,6674034,53386068
CGC_HOSP,53386068,<NA>,53386068,13505304,53386068
...,...,...,...,...,...
SIS_JUST,53386068,<NA>,<NA>,13523352,53386068
VAL_SH_FED,53386068,53386068,26693100,13368624,53386068
VAL_SP_FED,53386068,53386068,26693100,13386112,53386068


In [9]:
memoryUsage.to_csv('Types/Tipos.csv')

---

## Individual column study

#### UF_ZI

In [6]:
col = 'UF_ZI'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min    110000
max    530000
Name: UF_ZI, dtype: int64

In [7]:
col = 'UF_ZI'

# int 
df[col] = df[col].astype(np.int32)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_INT'] = a
print(f'Memory used for int: {a}')

# float
df[col] = df[col].astype(np.float32)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a
print(f'Memory used for flo: {a}')

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a
print(f'Memory used for cat: {a}')

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a
print(f'Memory used for obj: {a}')

Memory used for int: 26693100
Memory used for flo: 26693100
Memory used for cat: 13402856
Memory used for obj: 53386068


In [8]:
memoryUsage.loc['UF_ZI']

Espaço_CSV    53386068
Test_INT      26693100
Test_FLOAT    26693100
Test_CAT      13402856
Test_OBJ      53386068
Name: UF_ZI, dtype: object

In [9]:
df['UF_ZI'] = df['UF_ZI'].astype("category")

#### ANO_CMPT

In [10]:
col = 'ANO_CMPT'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min    2012
max    2012
Name: ANO_CMPT, dtype: int64

In [11]:
col = 'ANO_CMPT'

# int 
df[col] = df[col].astype(np.int16)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_INT'] = a

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a

# float
df[col] = df[col].astype(np.float16)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a

memoryUsage.loc[col]

Espaço_CSV    53386068
Test_INT      13346616
Test_FLOAT    13346616
Test_CAT       6673460
Test_OBJ      53386068
Name: ANO_CMPT, dtype: object

In [12]:
df['ANO_CMPT'] = df['ANO_CMPT'].astype("category")

#### MES_CMPT

In [13]:
col = 'MES_CMPT'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min     1
max    12
Name: MES_CMPT, dtype: int64

In [14]:
col = 'MES_CMPT'

# int 
df[col] = df[col].astype(np.int8)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_INT'] = a

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a

# float
df[col] = df[col].astype(np.float16)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a

memoryUsage.loc[col]

Espaço_CSV    53386068
Test_INT       6673374
Test_FLOAT    13346616
Test_CAT       6673574
Test_OBJ      53386068
Name: MES_CMPT, dtype: object

In [15]:
df['MES_CMPT'] = df['MES_CMPT'].astype(np.int8)

#### ESPEC

In [16]:
col = 'ESPEC'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min     1
max    14
Name: ESPEC, dtype: int64

In [17]:
col = 'ESPEC'

# int 
df[col] = df[col].astype(np.int8)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_INT'] = a

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a

# float
df[col] = df[col].astype(np.float16)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a

memoryUsage.loc[col]

Espaço_CSV    53386068
Test_INT       6673374
Test_FLOAT    13346616
Test_CAT       6673719
Test_OBJ      53386068
Name: ESPEC, dtype: object

In [18]:
df['ESPEC'] = df['ESPEC'].astype(np.int8)

#### CGC_HOSP

In [20]:
col = 'CGC_HOSP'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 1583013 out of 6673242 rows


min    1.508100e+10
max    9.871488e+13
Name: CGC_HOSP, dtype: float64

In [21]:
len(df['CGC_HOSP'].value_counts())

3319

In [22]:
col = 'CGC_HOSP'

# int 
if df[col].isna().sum() == 0:
    df[col] = df[col].astype(np.int8)

    a = df[col].memory_usage()
    memoryUsage.loc[col, 'Test_INT'] = a

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a

# float
df[col] = df[col].astype(np.float64)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a

memoryUsage.loc[col]

Espaço_CSV    53386068
Test_INT          <NA>
Test_FLOAT    53386068
Test_CAT      13505304
Test_OBJ      53386068
Name: CGC_HOSP, dtype: object

In [ ]:
df['CGC_HOSP'] = df['CGC_HOSP'].astype("category")

#### N_AIH

In [23]:
col = 'N_AIH'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min    1111100015388
max    9912300083587
Name: N_AIH, dtype: int64

In [24]:
col = 'N_AIH'

# int 
if df[col].isna().sum() == 0:
    df[col] = df[col].astype(np.int64)

    a = df[col].memory_usage()
    memoryUsage.loc[col, 'Test_INT'] = a

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a

# float
df[col] = df[col].astype(np.float64)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a

memoryUsage.loc[col]

Espaço_CSV     53386068
Test_INT       53386068
Test_FLOAT     53386068
Test_CAT      349677452
Test_OBJ       53386068
Name: N_AIH, dtype: object

In [25]:
df[col] = df[col].astype(np.int64)

#### IDENT

In [26]:
col = 'IDENT'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min    1
max    5
Name: IDENT, dtype: int64

In [27]:
col = 'IDENT'

# int 
if df[col].isna().sum() == 0:
    df[col] = df[col].astype(np.int8)

    a = df[col].memory_usage()
    memoryUsage.loc[col, 'Test_INT'] = a

# cat
df[col] = df[col].astype("category")

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_CAT'] = a

# float
df[col] = df[col].astype(np.float16)

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_FLOAT'] = a

# object
df[col] = df[col].astype('object')

a = df[col].memory_usage()
memoryUsage.loc[col, 'Test_OBJ'] = a

memoryUsage.loc[col]

Espaço_CSV    53386068
Test_INT       6673374
Test_FLOAT    13346616
Test_CAT       6673456
Test_OBJ      53386068
Name: IDENT, dtype: object

In [28]:
df[col] = df[col].astype(np.int8)

#### CEP

In [41]:
col = 'CEP'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min     1001000
max    99990000
Name: CEP, dtype: int64

In [62]:
retorno = type_test(df, memoryUsage, 'CEP')
print(retorno)
memoryUsage.loc['CEP']


int32


Espaço_CSV    53386068
Test_INT      26693100
Test_FLOAT    26693100
Test_CAT      41147920
Test_OBJ      53386068
Name: CEP, dtype: object

In [63]:
df['CEP'] = df['CEP'].astype(retorno)

In [64]:
df['CEP'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6673242 entries, 0 to 6673241
Series name: CEP
Non-Null Count    Dtype
--------------    -----
6673242 non-null  int32
dtypes: int32(1)
memory usage: 25.5 MB


#### MUNIC_RES

In [65]:
col = 'MUNIC_RES'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min    110001
max    530180
Name: MUNIC_RES, dtype: int64

In [67]:
retorno = type_test(df, memoryUsage, 'MUNIC_RES')
print(retorno)
memoryUsage.loc['MUNIC_RES']

category


Espaço_CSV    53386068
Test_INT      26693100
Test_FLOAT    26693100
Test_CAT      13523416
Test_OBJ      53386068
Name: MUNIC_RES, dtype: object

In [68]:
df['MUNIC_RES'] = df['MUNIC_RES'].astype(retorno)

#### NASC

In [69]:
col = 'NASC'

print(f'Null count: {df[col].isna().sum()} out of {len(df[col])} rows')
df[col].agg(['min', 'max'])

Null count: 0 out of 6673242 rows


min    19010117
max    20121230
Name: NASC, dtype: int64

In [72]:
type_test(df, memoryUsage, 'NASC')
memoryUsage.loc['NASC']

Espaço_CSV    53386068
Test_INT      26693100
Test_FLOAT    26693100
Test_CAT      27641116
Test_OBJ      53386068
Name: NASC, dtype: object

In [ ]:
df.NASC.info()

---

In [ ]:
'NASC', 'SEXO', 'UTI_MES_IN', 'UTI_MES_AN',
'UTI_MES_AL', 'UTI_MES_TO', 'MARCA_UTI', 'UTI_INT_IN', 'UTI_INT_AN',
'UTI_INT_AL', 'UTI_INT_TO', 'DIAR_ACOM', 'QT_DIARIAS', 'PROC_SOLIC',
'PROC_REA', 'VAL_SH', 'VAL_SP', 'VAL_SADT', 'VAL_RN', 'VAL_ACOMP',
'VAL_ORTP', 'VAL_SANGUE', 'VAL_SADTSR', 'VAL_TRANSP', 'VAL_OBSANG',
'VAL_PED1AC', 'VAL_TOT', 'VAL_UTI', 'US_TOT', 'DT_INTER', 'DT_SAIDA',
'DIAG_PRINC', 'DIAG_SECUN', 'COBRANCA', 'NATUREZA', 'NAT_JUR', 'GESTAO',
'RUBRICA', 'IND_VDRL', 'MUNIC_MOV', 'COD_IDADE', 'IDADE', 'DIAS_PERM',
'MORTE', 'NACIONAL', 'NUM_PROC', 'CAR_INT', 'TOT_PT_SP', 'CPF_AUT',
'HOMONIMO', 'NUM_FILHOS', 'INSTRU', 'CID_NOTIF', 'CONTRACEP1',
'CONTRACEP2', 'GESTRISCO', 'INSC_PN', 'SEQ_AIH5', 'CBOR', 'CNAER',
'VINCPREV', 'GESTOR_COD', 'GESTOR_TP', 'GESTOR_CPF', 'GESTOR_DT',
'CNES', 'CNPJ_MANT', 'INFEHOSP', 'CID_ASSO', 'CID_MORTE', 'COMPLEX',
'FINANC', 'FAEC_TP', 'REGCT', 'RACA_COR', 'ETNIA', 'SEQUENCIA',
'REMESSA', 'AUD_JUST', 'SIS_JUST', 'VAL_SH_FED', 'VAL_SP_FED',
'VAL_SH_GES', 'VAL_SP_GES'